# Disaster Tweet : True or False event ?

In this notebook, we are gonna show if a tweet concern a true or false event.\
We use Google trax.  

## Packages
Let's import packages we need


In [ ]:
# coding : utf8
!pip -q install trax
!pip -q install Flask

import numpy as np
import pandas as pd
from unicodedata import normalize
import trax
from trax.fastmath import numpy as fastnp
import trax.layers as tl
from trax.supervised import training 

     |████████████████████████████████| 471kB 5.7MB/s 
     |████████████████████████████████| 2.6MB 15.8MB/s 
     |████████████████████████████████| 174kB 36.0MB/s 
     |████████████████████████████████| 3.7MB 33.9MB/s 
     |████████████████████████████████| 71kB 5.1MB/s 
     |████████████████████████████████| 1.1MB 49.9MB/s 
     |████████████████████████████████| 348kB 43.0MB/s 
     |████████████████████████████████| 1.3MB 48.9MB/s 
     |████████████████████████████████| 2.9MB 43.7MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
ERROR: transformers 3.5.1 has requirement sentencepiece==0.1.91, but you'll have sentencepiece 0.1.94 which is incompatible.


## Dataset
We use a dataset from a kaggle competition

In [ ]:
# Data importation
data = pd.read_csv('train.csv', encoding = "utf8")
print(data.head())

   id keyword  ...                                               text target
0   1     NaN  ...  Our Deeds are the Reason of this #earthquake M...      1
1   4     NaN  ...             Forest fire near La Ronge Sask. Canada      1
2   5     NaN  ...  All residents asked to 'shelter in place' are ...      1
3   6     NaN  ...  13,000 people receive #wildfires evacuation or...      1
4   7     NaN  ...  Just got sent this photo from Ruby #Alaska as ...      1

[5 rows x 5 columns]


In [ ]:
# Description of our Data.
print(data.describe())
print("Count values of target")
print(data.target.value_counts())

                 id      target
count   7613.000000  7613.00000
mean    5441.934848     0.42966
std     3137.116090     0.49506
min        1.000000     0.00000
25%     2734.000000     0.00000
50%     5408.000000     0.00000
75%     8146.000000     1.00000
max    10873.000000     1.00000
Count values of target
0    4342
1    3271
Name: target, dtype: int64


In [ ]:
# Create a list of tupe(sentence, target)
from unicodedata import normalize


def create_list_of_tuple(data, feature, target) : 
  """
  This function trasnform a data in list of tuple(feature, target)
  input : data : a pandas dataframe
  feature : feature : columns of data we need
  target : target
  output : t tuple(sentence, target)
  """
  #if isinstance(data, pd.DataFrame) : 
  assert type(data) == pd.core.frame.DataFrame, 'Enter pandas dataframe '
  assert type(feature) == str, 'Enter a list of feature'
  assert type(target) == str, 'Enter a list of target'

  t = []
  for x,y in zip(data[feature].iloc, data[target].iloc): 
    t.append((x, y))
  return t

In [ ]:
data_l = create_list_of_tuple(data, "text", "target")
print("Twio first line or data are : ", data_l[:2])

Twio first line or data are :  [('Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all', 1), ('Forest fire near La Ronge Sask. Canada', 1)]


## Create a vocabulary
Before creating of vocabulary, we will process our data. 
We will create some functions. 
process_tweet


In [114]:
import re
import nltk
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words("english")
from nltk.stem import PorterStemmer


def process_tweet(tweet) : 
  stem = PorterStemmer()
  assert type(tweet) == str, "Input must be string"
  tweet = tweet.lower()

  #tweet = re.sub(r'https?:\/\/\w+', '<link> ', tweet)
  tweet = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '__link__', tweet)
  # remove hashtags
  # only removing the hash # sign from the word
  tweet = re.sub(r'#|@', '', normalize("NFKC", tweet))

  tweet = re.sub(r'[\[\]|\-!?:.,\'()]', '', tweet)

  tweet_clean = word_tokenize(tweet)
  tweet_clean = [stem.stem(x) for x in tweet_clean if x not in sw]

  return tweet_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Create a vocabulary from sentencepiece


In [115]:
sent = "320 [IR] ICEMOON [AFTERSHOCK] the | http://t.co/vAM5POdGyw | @djicemoon | #Dubstep #TrapMusic #DnB #EDM #Dance #IcesÛ_ http://t.co/zEVakJaPcz lonely"
process_tweet(sent)

['320',
 'ir',
 'icemoon',
 'aftershock',
 '__link__',
 'djicemoon',
 'dubstep',
 'trapmus',
 'dnb',
 'edm',
 'danc',
 'ices\x89û_',
 '__link__',
 'lone']

In [ ]:
import csv
# Process tweets
#from collections import defaultdict
def get_dict(data_l) : 
  tokenized_l = []
  word2ind = {"__unk__" : 1, "__pad__" : 0}
  for tweet,target in data_l : 
    tokens = process_tweet(tweet)
    tokenized_l.append((tweet, target))

    # Create a word2ind dictionary
    for word in tokens : 
      if word not in word2ind.keys() : 
        word2ind[word] = len(word2ind)

  return word2ind

# word2ind dictionary
word2ind = get_dict(data_l)
print(f"Vocab size is : {len(word2ind)}")



Vocab size is  :15189


Define a tweet to tensor function

In [ ]:
def tweet_to_tensor(tweet, unknown , vocab_dict) : 
  """
   This function trasnform a tweet to a tensor
   Input : 
   tweet  : String
   unknown : symbol of unknown word
   vocab_dict :  a dictionary which map word to indice
   output : 
   tensor  : list/array
   """
  tokenized = process_tweet(tweet)

  unk_id = vocab_dict[unknown]

  tensor = []
  for token in tokenized : 
    if token in vocab_dict.keys() : 
      tensor.append(vocab_dict[token])
    else : 
      tensor.append(unk_id)
  
  return tensor


Define a data 
generator

In [84]:
import random

def data_generator(data, batch_size, loop, vocab_dict, shuffle = False) : 

  #assert batch_size %2 == 0, "batch size must be odd"

  # Set index to 0
  index = 0

  # Get len data
  len_data = len(data)
  
  # Get and array with the data indexes
  index_lines = list(range(len_data))
  
  # shuffle lines if shuffle is set to True
  if shuffle:
      random.shuffle(index_lines)

  stop = False

  # Loop indefintively
  while not stop :

    batch = [] # Create batch
    target_l = [] # Create target

    for i in range(batch_size) : 

      if index>=len_data : 

        # If loop is set to False, break once we reach the end of the dataset
        if not loop : 
          stop = True
          break

        index = 0
        # If user wants to keep re-using the data, reset the index
        if shuffle : 
          random.shuffle(index_lines)


      tweet = data[index_lines[index]][0]
      target = data[index_lines[index]][1]

      # convert the tweet into tensors of integers representing the processed words
      tensor = tweet_to_tensor(tweet,'__unk__', vocab_dict)
      #tensor = tokenize(tweet)

      # append the tensor to the batch list
      batch.append(tensor)
      target_l.append(target)
      
    # Increment pos_index by one
      index = index + 1

    if stop:
      break;

  # Update the start index for positive data 
  # so that it's n_to_take positions after the current pos_index
    index += batch_size

    # Get the max tweet length (the length of the longest tweet) 
    # (you will pad all shorter tweets to have this length)
    max_len = max([len(t) for t in batch]) 
    
    
    # Initialize the input_l, which will 
    # store the padded versions of the tensors
    tensor_pad_l = []
      # Pad shorter tweets with zeros
    for tensor in batch:
      n_pad = max_len - len(tensor)
        
      # concatenate the tensor and the list of padded zeros
      tensor_pad = tensor + [0] * n_pad
        
      # append the padded tensor to the list of padded tensors
      tensor_pad_l.append(tensor_pad)

      # convert the list of padded tensors to a numpy array
      # and store this as the model inputs
    inputs = np.array(tensor_pad_l)
      
      # Convert the target list into a numpy array
    targets = np.array(target_l)

      # Example weights: Treat all examples equally importantly.It should return an np.array. Hint: Use np.ones_like()
    example_weights = np.ones_like(targets)

    yield inputs, targets, example_weights



In [85]:
stream = data_generator(data_l, 32, True, word2ind, shuffle = True)

In [106]:
# Test data generator
input,target, mask = next(stream)

# Print shape of data generator outputs
print(input.shape, target.shape, mask.shape)

(32, 53) (32,) (32,)


In [118]:
# Eval and train stream
import random
def train_test_set(data_l, eval_size = 0.05) : 
  """
  input : text : list of tuple of english sentence and its translation in french
  output : train set and eval set
  """
  random.shuffle(data_l)
  length = len(data_l)
  cutoff = int(length * (1-eval_size))
  return data_l[:cutoff], data_l[cutoff:] 

# Get train and test set
train_set, eval_set = train_test_set(data_l, 0.15)

# print length of train and test set
print(len(train_set), len(eval_set))

6471 1142


In [116]:
# Create the training data generator
def train_generator(batch_size, shuffle = False):
    return data_generator(train_set, batch_size, True, word2ind, shuffle)

# Create the validation data generator
def val_generator(batch_size, shuffle = False):
    return data_generator(eval_set, batch_size, True, word2ind, shuffle)

In [119]:
def classifier(vocab_size=len(word2ind), embedding_dim=2048, output_dim=2, mode='train'):
        
### START CODE HERE (Replace instances of 'None' with your code) ###
    # create embedding layer
    embed_layer = tl.Embedding(
        vocab_size=vocab_size, # Size of the vocabulary
        d_feature=embedding_dim)  # Embedding dimension
    
    # Create a mean layer, to create an "average" word embedding
    mean_layer = tl.Mean(axis = 1, keepdims= False)

    lstm = tl.LSTM(embedding_dim)
    
    # Create a dense layer, one unit for each output
    dense_output_layer = tl.Dense(n_units = output_dim)

    
    # Create the log softmax layer (no parameters needed)
    log_softmax_layer = tl.LogSoftmax()
    
    # Use tl.Serial to combine all layers
    # and create the classifier
    # of type trax.layers.combinators.Serial
    model = tl.Serial(
      embed_layer, # embedding layer
      mean_layer, # mean layer
      dense_output_layer, # dense output layer 
      log_softmax_layer # log softmax layer
    )
### END CODE HERE ###     
    
    # return the model of type
    return model

In [120]:
batch_size = 64

# Create a train and test task for training
train_task = training.TrainTask(
    labeled_data=train_generator(batch_size=batch_size, shuffle=True),
    loss_layer=tl.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.0001),
    n_steps_per_checkpoint=10,
)

eval_task = training.EvalTask(
    labeled_data=val_generator(batch_size=batch_size, shuffle=True),
    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()])

In [109]:
import os
output_dir = '/model/'
output_dir_expand = os.path.expanduser(output_dir)
print(output_dir_expand)

# UNQ_C6 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: train_model
def train_model(classifier, train_task, eval_task, n_steps, output_dir):
    '''
    Input: 
        classifier - the model you are building
        train_task - Training task
        eval_task - Evaluation task
        n_steps - the evaluation steps
        output_dir - folder to save your files
    Output:
        trainer -  trax trainer
    '''
### START CODE HERE (Replace instances of 'None' with your code) ###
    training_loop = training.Loop(
                                classifier, # The learning model
                                train_task, # The training task
                                eval_tasks = eval_task, # The evaluation task
                                output_dir = output_dir) # The output directory

    training_loop.run(n_steps = n_steps)
### END CODE HERE ###

    # Return the training_loop, since it has the model.
    return training_loop

/model/


In [92]:
model = classifier()
print(model)

Serial[
  Embedding_15189_2048
  Mean
  Dense_2
  LogSoftmax
]


In [122]:

training_loop = train_model(model, train_task, eval_task, 100, '.')


Step    210: Ran 10 train steps in 20.56 secs
Step    210: train CrossEntropyLoss |  0.64818811
Step    210: eval  CrossEntropyLoss |  0.66104668
Step    210: eval          Accuracy |  0.57812500

Step    220: Ran 10 train steps in 18.29 secs
Step    220: train CrossEntropyLoss |  0.65991086
Step    220: eval  CrossEntropyLoss |  0.65271246
Step    220: eval          Accuracy |  0.57812500

Step    230: Ran 10 train steps in 18.22 secs
Step    230: train CrossEntropyLoss |  0.65608096
Step    230: eval  CrossEntropyLoss |  0.64291835
Step    230: eval          Accuracy |  0.67187500

Step    240: Ran 10 train steps in 17.94 secs
Step    240: train CrossEntropyLoss |  0.66375715
Step    240: eval  CrossEntropyLoss |  0.65520656
Step    240: eval          Accuracy |  0.59375000

Step    250: Ran 10 train steps in 18.94 secs
Step    250: train CrossEntropyLoss |  0.65612048
Step    250: eval  CrossEntropyLoss |  0.68603837
Step    250: eval          Accuracy |  0.54687500

Step    260: R

In [ ]:
# Create a generator object
tmp_train_generator = train_generator(16)

# get one batch
tmp_batch = next(tmp_train_generator)

# Position 0 has the model inputs (tweets as tensors)
# position 1 has the targets (the actual labels)
tmp_inputs, tmp_targets, tmp_example_weights = tmp_batch

print(f"The batch is a tuple of length {len(tmp_batch)} because position 0 contains the tweets, and position 1 contains the targets.") 
print(f"The shape of the tweet tensors is {tmp_inputs.shape} (num of examples, length of tweet tensors)")
print(f"The shape of the labels is {tmp_targets.shape}, which is the batch size.")
print(f"The shape of the example_weights is {tmp_example_weights.shape}, which is the same as inputs/targets size.")


# feed the tweet tensors into the model to get a prediction
tmp_pred = training_loop.eval_model(tmp_inputs)
print(f"The prediction shape is {tmp_pred.shape}, num of tensor_tweets as rows")
print("Column 0 is the probability of a negative sentiment (class 0)")
print("Column 1 is the probability of a positive sentiment (class 1)")
print()
print("View the prediction array")
tmp_pred

The batch is a tuple of length 3 because position 0 contains the tweets, and position 1 contains the targets.
The shape of the tweet tensors is (16, 26) (num of examples, length of tweet tensors)
The shape of the labels is (16,), which is the batch size.
The shape of the example_weights is (16,), which is the same as inputs/targets size.
The prediction shape is (16, 2), num of tensor_tweets as rows
Column 0 is the probability of a negative sentiment (class 0)
Column 1 is the probability of a positive sentiment (class 1)

View the prediction array


DeviceArray([[-0.02711511, -3.6211896 ],
             [-4.0115175 , -0.01827192],
             [-0.02149475, -3.8506737 ],
             [-0.20396167, -1.6900711 ],
             [-0.41540757, -1.0790191 ],
             [-0.10753   , -2.2832696 ],
             [-0.50392365, -0.92673385],
             [-0.8582263 , -0.55149704],
             [-1.5595372 , -0.2360177 ],
             [-0.08233225, -2.5378766 ],
             [-0.01742125, -4.0587654 ],
             [-0.03321397, -3.4213448 ],
             [-3.3148766 , -0.0370152 ],
             [-0.7447773 , -0.6440523 ],
             [-0.11672461, -2.2057328 ],
             [-1.7174097 , -0.19787866]], dtype=float32)

In [ ]:
# turn probabilites into category predictions
tmp_is_positive = tmp_pred[:,1] > tmp_pred[:,0]
for i, p in enumerate(tmp_is_positive):
    print(f"Neg log prob {np.exp(tmp_pred[i,0]):.4f}\tPos log prob {np.exp(tmp_pred[i,1]):.4f}\t is positive? {p}\t actual {tmp_targets[i]}")


Neg log prob 0.9732	Pos log prob 0.0268	 is positive? False	 actual 0
Neg log prob 0.0181	Pos log prob 0.9819	 is positive? True	 actual 1
Neg log prob 0.9787	Pos log prob 0.0213	 is positive? False	 actual 0
Neg log prob 0.8155	Pos log prob 0.1845	 is positive? False	 actual 1
Neg log prob 0.6601	Pos log prob 0.3399	 is positive? False	 actual 1
Neg log prob 0.8980	Pos log prob 0.1020	 is positive? False	 actual 0
Neg log prob 0.6042	Pos log prob 0.3958	 is positive? False	 actual 0
Neg log prob 0.4239	Pos log prob 0.5761	 is positive? True	 actual 1
Neg log prob 0.2102	Pos log prob 0.7898	 is positive? True	 actual 1
Neg log prob 0.9210	Pos log prob 0.0790	 is positive? False	 actual 0
Neg log prob 0.9827	Pos log prob 0.0173	 is positive? False	 actual 1
Neg log prob 0.9673	Pos log prob 0.0327	 is positive? False	 actual 0
Neg log prob 0.0363	Pos log prob 0.9637	 is positive? True	 actual 1
Neg log prob 0.4748	Pos log prob 0.5252	 is positive? True	 actual 0
Neg log prob 0.8898	Pos l

In [ ]:
predict = classifier()
# instantiate the model we built in eval mode
predict = classifier(mode = "eval")
 
# initialize weights from a pre-trained model
predict.init_from_file("model.pkl.gz", weights_only=True)
predict = tl.Accelerate(model)

In [ ]:
modele = classifier()
modele.init_from_file('model.pkl.gz')

def prediction(sentence, model = model, vocab_dict = word2ind) :

  input = np.array(tweet_to_tensor(sentence, '__unk__', vocab_dict))[None, :]

  preds = model(input)
  classification = np.argmax(preds)
  prob = np.exp(preds[0, classification])

  return abs(1-classification), prob 

Prediction for test few sentence of test set

In [ ]:
test = pd.read_csv("test.csv")
sentences = list(np.squeeze(test.text))
random.shuffle(sentences)
i = 0
for sentence in sentences : 
  i += 1
  print(sentence, prediction(sentence))
  if i > 100 : 
    break

Wildfire near Columbia River town is 50 percent contained - http://t.co/yTPiPXpqr9 http://t.co/HCUQ6jpBtL (0, 0.9975071)
.@MartinMJ22 @YouGov Which '#Tory landslide' ... you can't POSSIBLY mean the wafer-thin majority of #GE2015?!! (1, 0.7468)
Amazing how smug this guy is about the use of mass murder as a global power play. https://t.co/AlTTBJy8B9 (1, 0.7391276)
IF SUICIDE BOMBING WASTHE SMARTEST THING2 DO FOR ALLAH/GODJESUS/THE HOLY PROPHET MUHAMMAD COULD HAVE KILLEDSOMEBODY? http://t.co/tGfWuVVHxj (0, 0.9681884)
Waige driving like a Badass #ScorpionPilot #Cyclone @ScorpionCBS (0, 0.79837376)
@Riot_Sweet @jfwong @MtgoDoc @JoshLeeKwai wait.. what dates?? ill be at MSG and PAX =( (1, 0.87318)
I BOMBED THE FAKE BOLT CHAT AND EVERYONE THOUGHT BOLT ENDED LMFAO (1, 0.5799052)
If our love is tragedy why are you my remedyyyy (1, 0.9979462)
I got more rhymes than the Bible's got psalms
And just like the Prodigal Son I've returned
Anyone stepping to me you'll get burned
?? (1, 0.96373075)
Inves